# Исследование надежности заемщиков.

**Описание проекта**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

##  Обзор данных.

In [1]:
import pandas as pd  # импорт библиотеки pandas
data = pd.read_csv('C:/Users/Анастасия/Documents/Jupiter_project/projects/data_preprocessing/data.csv') # чтение файла с данными и сохранение в data
display(data.head(10)) # получение первых 10 строк таблицы data
data.info() # получение общей информации о данных в таблице data
data.columns # перечень названий столбцов таблицы data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

В таблице 11 столбцов:

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

Выявленные проблемы:
* 1. Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.
* 2. В первых же строках заметны аномальные отрицательные значения в столбце с трудовым стажем, а также аномально большое значение в строке 4.
* 3. В столбцах `days_employed` и `total_income` тип данных для удобства и корректного отображения лучше изменить с float на int, т.к. точность до миллисекунд в трудовом стаже и до копейки в доходе вряд ли нам потребуется, а таблица станет выглядеть значительно лучше и проще для восприятия.
* 4. В столбце с образованием заметна разная литерация в значениях, что приведет к дубликатам. Также в столбце с целью кредита видны похожие словоформы и формулирвоки, которые являются дубликатами по сути и их можно было бы привести к одному виду.

Названия столбцов в приемлемом формате. Необходимо устранить выявленные проблемы, а также подробнее посмотреть на значения в таблице для устранения возможных дубликатов и аномалий.

## Заполнение пропусков

In [2]:
data.isna().sum() # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [3]:
data.isna().mean() #подсчет долей пропусков

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Пропущенные значения находятся в столбцах с общим трудовым стажем и ежемесячным доходом, они являются количественными. Данные пропуски лучшего всего заменить медианным значением тк разброс максимальных и минимальных значений в них большой. Среднее значение не будет давать достоверную информацию для исследования.

К замене значений столбца `days_employed` лучше вернемся после исправления аномальных значений в нем, чтобы расчет оказался корректным. 

Рассмотрим столбец `total_income` и пропуски данных в нем. Пропущено 2174 значения, общее количество строк 21525, итого около 10% значений. Это достаточно большое количество данных, поэтому удалять строки с пустыми значения в этом солбце мы точно не будем. Также необходимо посмотреть какие именно это строки, тк по количеству пропусков видим подозрительное совпадение с количеством пропусков с столбце со стажем, не исключено, что это пропуски в одних и тех же строках.

In [4]:
data[data['total_income'].isna()].head(20) #просмотр строк с пропусками значений

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


При анализе данных строк, видим, что действительно, пропущенные данные одновремнно появляются в двух столбцах. При этом пропуски появляются вне зависимости от значений в других столбцах, практически во всех возможных категориях заемщиков. Либо некорректная выгрузка стала причиной, либо ошибка при заполении анкеты заемщиком, либо просто его нежелание указывать необходимую информацию о себе, возможно, без официального трудоустройства.

На всякий случай сразу проверим столбец `total_income` на аномальные значения, чтобы получить адекватные медаины для его заполнения.

In [5]:
round(data['total_income'].describe(), 2)


count      19351.00
mean      167422.30
std       102971.57
min        20667.26
25%       103053.15
50%       145017.94
75%       203435.07
max      2265604.03
Name: total_income, dtype: float64

В целом, по данным видно, что явных аномалий нет, каких-то отрицательных, нулевых или слишком огромных значений нет, медиана "похожа на правду".

В реальности значение дохода было бы достаточно принципиальным показателем и количество пропусков велико, чтобы обратиться к программистам или источнику информации, чтобы уточнить недостающие данные или получить информацию о причинах ее остутствия, чтобы приянть решение о дальнейшем использовании ее в исследовании. В данной учебном проекте заполним пропуски данными медианных значенией по категориям занятости, предполагая, что внутри групп с одинаковым типом занятости будут максимально приближенные данные.

In [6]:
# получим список типов занятости, в которых есть пропуски
list_type = [] #создание списка для категорий, в которых пропущены значения
for i in data['income_type'].unique():
    if data[data['income_type']== i]['total_income'].isna().sum() > 0:
        list_type.append(i)
        print(i, data[data['income_type']== i]['total_income'].isna().sum())      

сотрудник 1105
пенсионер 413
компаньон 508
госслужащий 147
предприниматель 1


Видим, что необходимо получить расчет медиан для категорий: сотрудник, пенсионер, компаньон, госслужащий, предприниматель. При этом рассмотрим наполненность этих категорий.

In [7]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Для всех кроме предпринимателя данных вполне достаточно для рассчета медианы. Но одного значения точно недостаточно, предприниматели могут иметь совершенно разных доход. Считаю, что данную строку просто исключить из исследования, предприниматель с неизвестным доходом будет удален для объективности.

In [8]:
data[data['income_type']=='предприниматель'] #посмотрим на предпринимателей

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
18697,0,-520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы


In [9]:
data=data.drop(index=[5936]).reset_index(drop=True) #удалим строку по номеру и предпринимателя из списка
list_type.remove('предприниматель')
data[data['income_type']=='предприниматель'] #проверка

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
18696,0,-520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы


Осталось разобраться с четырьмя группами, посчитаем медианные значения в них и присвоим пропускам соотвествующие значения.

In [10]:
median_data = data.groupby('income_type')['total_income'].median() #определение медианного дохода по категориям
for category in list_type:                                         #замена пустых ячеек медианным доходом по данной категории
    median_income = median_data.loc[category]
    data.loc[data['income_type']==category, 'total_income'] = \
    data.loc[data['income_type']==category, 'total_income'].fillna(median_income)

Проверим на всякий случай первую строку с пропуском значения, в которой находился пенсионер со значением медианы. Значение совпадает.

In [11]:
print(median_data)
data.loc[12]

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


children                           0
days_employed                    NaN
dob_years                         65
education                    среднее
education_id                       1
family_status       гражданский брак
family_status_id                   1
gender                             M
income_type                пенсионер
debt                               0
total_income           118514.486412
purpose              сыграть свадьбу
Name: 12, dtype: object

## Проверка данных на аномалии и исправления.

Рассмотрим подробнее остальные столбцы для обнаружения аномальных значений

### children - количество детей

In [12]:
data['children'].value_counts()  #подсчет уникальных значений в столбце

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Видим, что в целом данные выглядят адекватно. Но есть 47 значений -1 - количество детей не могло быть отрицательным, и 76 значений 20, возможно, но маловероятно. Скорее всего -1 могло быть равно 1, а 20 это 2, но наверняка нельзя сказать. Суммарно аномальных значений менее 1%. Поэтому, думаю, что тк исследование непосрдественно затрагивает взаимосвязь с количеством детей, эти данные лучше исключить из исследования.

In [13]:
data = data[(data['children']!=-1)&(data['children']!=20)] #удаляем строки со значением детей -1 и 20
data.info() #проверка

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21401 entries, 0 to 21523
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21401 non-null  int64  
 1   days_employed     19240 non-null  float64
 2   dob_years         21401 non-null  int64  
 3   education         21401 non-null  object 
 4   education_id      21401 non-null  int64  
 5   family_status     21401 non-null  object 
 6   family_status_id  21401 non-null  int64  
 7   gender            21401 non-null  object 
 8   income_type       21401 non-null  object 
 9   debt              21401 non-null  int64  
 10  total_income      21401 non-null  float64
 11  purpose           21401 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### days_employed - дни стажа

In [14]:
data['days_employed'].describe() #рассмотрим данные по разбросу значений в столбце

count     19240.000000
mean      63159.820777
std      140928.943329
min      -18388.949901
25%       -2747.235601
50%       -1203.934202
75%        -289.740178
max      401755.400475
Name: days_employed, dtype: float64

По данной таблице видно, что значения о днях трудового стажа вряд ли можно назвать достоверными. Присутствуют отрицательные значения, причем в таком количестве, что медианное значение имеет отрицательный знак. Но и максимальное значение не похоже на правду, если перевести 401755 в года это 1100 лет. 

Если бы в нашем исследовании данный фактор имел значение, скорее всего требовлаось бы уточнить информацию или провести целое расследование о причинах возникновения ошибок и что именно "пошло не так" при записи этого столбца. Можно лишь выдвинуть гипотезы: 

* что "-" это на самом деле точно такое же значение, но с "+", 
* слишком большие значения переведены в часы, а не дни
* стаж рассчитывался по формуле и при внесении данных возникла ошибка в годах, в пользу этой версии говорит то, что дата могла записываться в формате 01.01.22 и 01.01.79, при этом имелся ввиду 1979 и 2022 год, а рассчет мог быть построен на 2079 и 2022 или наоборот 1979 и 1922. 
* стаж рассчитывался по формуле и при внесении данных ошибка возникла в том, что были перепутаны ячейки с вводимой информацией, те дата начала работы и последнего места работы скажем оказались перепутаны.


Но в действиетелньости это гадание на кофейной гуще без наличия дополнительной инфорамации и в данном проекте трудовой стаж не является необходимым показателем, поэтому данный столбец можно просто удалить из таблицы без потери чего-либо ценного.

In [15]:
data.pop('days_employed')  #удаление столбца и проверка
data.columns

Index(['children', 'dob_years', 'education', 'education_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

### dob_years - возраст

In [16]:
#просмотр информации о значениях в столбце
data['dob_years'].value_counts().sort_index()

0     100
19     14
20     51
21    110
22    183
23    252
24    263
25    356
26    406
27    490
28    501
29    543
30    536
31    556
32    506
33    577
34    597
35    614
36    553
37    531
38    595
39    572
40    603
41    603
42    592
43    510
44    543
45    494
46    469
47    480
48    536
49    505
50    509
51    446
52    483
53    457
54    476
55    441
56    482
57    457
58    460
59    441
60    376
61    353
62    351
63    268
64    263
65    194
66    183
67    167
68     99
69     83
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

Возраст 0 явная аномалия, в остальном отклонений от нормы не видно. Проверим строки, в которых встречается данное значение.

In [17]:
data[data['dob_years']==0].head(15)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,142594.396847,жилье
1898,0,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


Суммарно 100 аномальных значений - менее 0,5%. При этом взаимосявязи между пропусками нет, они возникли в совершенно разных категориях заемщиков. Данное исследование не затрагивает возраст заемщиков, при этом информация в других столбцах может быть полезной, поэтому данные "0" можно оставить на месте, на исследование они не повлияют.

### education, education_id, family_status, family_status_id

In [18]:
data['education'].value_counts()

среднее                13667
высшее                  4697
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [19]:
data['education_id'].value_counts()

1    15136
0     5236
2      741
3      282
4        6
Name: education_id, dtype: int64

5 типов образования соответствуют 5 id, есть дубликаты, но аномалий нет

In [20]:
data['family_status'].value_counts()

женат / замужем          12301
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

In [21]:
data['family_status_id'].value_counts()

0    12301
1     4160
4     2799
3     1189
2      952
Name: family_status_id, dtype: int64

5 типов статусов, 5 id, аномалий нет

### gender - пол

In [22]:
data['gender'].value_counts()

F      14154
M       7246
XNA        1
Name: gender, dtype: int64

Пол со значением "XNA" явная аномалия, посмотрим на строку, в которой она возникла.

In [23]:
data[data['gender']=='XNA']

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10700,0,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


В целом, по остальным данным невозможно определить какого пола был заемщик. Для исследования данная информация не важна, и не повлияет на выводы, но, чтобы привести пол к единому виду, можем заменить значение на F.

In [24]:
data.loc[data['gender']=='XNA','gender'] = 'F' #замена значения и проверка
data['gender'].value_counts()

F    14155
M     7246
Name: gender, dtype: int64

### income_type - тип занятости, debt - была ли задолженность

In [25]:
data['income_type'].value_counts()

сотрудник          11050
компаньон           5054
пенсионер           3839
госслужащий         1453
безработный            2
студент                1
предприниматель        1
в декрете              1
Name: income_type, dtype: int64

In [26]:
data['debt'].value_counts()

0    19669
1     1732
Name: debt, dtype: int64

Аномалий в столбцах нет

### purpose - цель кредита

In [27]:
data['purpose'].value_counts()

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка своего жилья                      619
покупка недвижимости                      619
ремонт жилью                              609
покупка жилой недвижимости                602
на покупку своего автомобиля              504
заняться высшим образованием      

Много разных словоформ-дубликатов, но аномалий нет. Данный столбец будем категоризировать с созданием нового для исследования.

## Изменение типов данных.

В столбце `total_income` приведем данные к типу int для удобства.

In [28]:
data['total_income'] = data['total_income'].astype(int)
data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21401 entries, 0 to 21523
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21401 non-null  int64 
 1   dob_years         21401 non-null  int64 
 2   education         21401 non-null  object
 3   education_id      21401 non-null  int64 
 4   family_status     21401 non-null  object
 5   family_status_id  21401 non-null  int64 
 6   gender            21401 non-null  object
 7   income_type       21401 non-null  object
 8   debt              21401 non-null  int64 
 9   total_income      21401 non-null  int32 
 10  purpose           21401 non-null  object
dtypes: int32(1), int64(5), object(5)
memory usage: 1.9+ MB


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


## Удаление дубликатов.

Просмотрим значения в столбце `education`, скорее всего при заполнении данных имел место быть просто человеческий фактор, поэтому они имеют всевозможный разный вид написания и являются неявнями дубликатами. Приведем их к единому регистру(нижнему).

In [29]:
data['education'].value_counts()

среднее                13667
высшее                  4697
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [30]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15136
высшее                  5236
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

Остальные столбцы были просмотрены в ходе поиска аномальных значений, в них неявных дубликатов больше нет.

Теперь проверим таблицу на наличие полных дубликатов строк и при необходимости удалим их.

In [31]:
data.duplicated().sum()

71

Был найден 71 дубликат, удаляем их и проверяем.

In [32]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

## Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Чтобы упростить работу с таблицей и ее визуализацию, создадим отдельные "словари", в которых:
* каждому уникальному значению из `education` соответствует уникальное значение `education_id` — в первом;
* каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id` — во втором.

In [33]:
education_dict = data[['education', 'education_id']] #создаем "словарь"
education_dict.head(10)

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
5,высшее,0
6,высшее,0
7,среднее,1
8,высшее,0
9,среднее,1


In [34]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)  #удаляем из него дубликаты
education_dict.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [35]:
family_status_dict = data[['family_status', 'family_status_id']]
family_status_dict.head(10)                          

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
5,гражданский брак,1
6,женат / замужем,0
7,женат / замужем,0
8,гражданский брак,1
9,женат / замужем,0


In [36]:
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)  #удаляем из него дубликаты
family_status_dict.head()

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Теперь удалим из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`.

In [37]:
data = data.drop(columns = ['education', 'family_status']) #удаление столбцов и проверка
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21330 entries, 0 to 21329
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21330 non-null  int64 
 1   dob_years         21330 non-null  int64 
 2   education_id      21330 non-null  int64 
 3   family_status_id  21330 non-null  int64 
 4   gender            21330 non-null  object
 5   income_type       21330 non-null  object
 6   debt              21330 non-null  int64 
 7   total_income      21330 non-null  int32 
 8   purpose           21330 non-null  object
dtypes: int32(1), int64(5), object(3)
memory usage: 1.4+ MB


,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


## Категоризация дохода.

На основании диапазонов, указанных ниже, создадим столбец `total_income_category` с категориями:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 100001 и выше — 'A'.

In [38]:
def income_category(income): #функция для определения категории дохода
    if 0 <= income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    if income >= 1000001:
        return 'A'

In [39]:
income_category(1000005) #проверка

'A'

In [40]:
data['income_category'] = data['total_income'].apply(income_category) #применяем функцию к столбцу total_income
data.tail(10)

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_category
21320,1,28,1,0,F,сотрудник,1,109486,заняться образованием,C
21321,0,42,0,0,F,компаньон,0,322807,покупка своего жилья,B
21322,0,42,0,1,F,компаньон,0,178059,на покупку своего автомобиля,C
21323,0,59,1,0,F,пенсионер,0,153864,сделка с автомобилем,C
21324,1,37,4,3,M,сотрудник,0,115949,покупка коммерческой недвижимости,C
21325,1,43,1,1,F,компаньон,0,224791,операции с жильем,B
21326,0,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21327,1,38,1,1,M,сотрудник,1,89672,недвижимость,C
21328,3,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B
21329,2,40,1,0,F,сотрудник,0,82047,на покупку автомобиля,C


## Категоризация целей кредита.

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец `purpose_category`, в который войдут следующие категории:
* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'.

Для этого проведем лемматизацию.

In [41]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [ ]:
from pymystem3 import Mystem
m = Mystem()
   
def purpose_category(purpose):   #создаем столбец с категоризированной целью кредита на основе лемм столбца purpose
    lemmas_row = m.lemmatize(purpose)
    for i in lemmas_row:
        if 'авто' in i:
            return 'операции с автомобилем'
        if 'недвиж' in i or 'жил' in i:
            return  'операции с недвижимостью'
        if 'свад' in i:
            return  'проведение свадьбы'
        if 'образ' in i:
            return 'получение образования'
        
data['purpose_category']= data['purpose'].apply(purpose_category)     #запускаем функцию для столбца puprpose   
data.head(10)

In [ ]:
data['purpose_category'].unique()

## Ответы на вопросы.

### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для ответа на данный вопрос предлагаю рассмотреть и сравнить процент просрочек при разных значениях количества детей. Сформируем сводную таблицу по общему количеству просроченных кредитов в зависимости от количества детей.

In [ ]:
data.pivot_table(index=['gender'], columns='children', values='debt', aggfunc='sum')

Т.к. для ответа на каждый из вопросов мы будем рассматривать один и тот же показатель, но в зависимости от разных переменных, напишем сразу функцию для расчета.

In [ ]:
def ratio_factor(factor): #где factor это столбец таблицы, взаимосявязь с которым мы хотим рассмотреть
    table_debt = data.groupby(factor).agg({'debt': ['sum', 'count']}) #считаем количество просрочек и общее количество кредитов и группируем по фактору
    table_debt['ratio'] = round(table_debt[('debt', 'sum')] / table_debt[('debt', 'count')]*100,2) #находим долю просроченных займов
    display(table_debt.sort_values(by='ratio', ascending=False))    #сортируем итоговую таблицу
    display(data[factor].value_counts(normalize=True)*100)          #доля значений в общем количестве

In [ ]:
ratio_factor('children')

Как видим из таблиц, количество займов, совершенных людьми с количеством детей от 3 до 5 довольно незначительное, чтобы на их основе делать какие-то выводы, посчитаем общий процент, приходящийся на все эти категории от общего числа.

Итого получаем, что на кредиты выданные многодетным семьям приходится меньше 2% от всего объема. Вполне естественно, что семей с бОльшим количеством детей в принципе меньше, возможно, они и реже обращаются за кредитами.

**Выводы:** 
* в категориях с количеством детей от 3 до 5 - не такая большая выборка, чтобы делать на их основе достоверные выводы, на них суммарно приходится меньше 2% всего объема кредитов. При более подробном исследовании, возможно, имело бы смысл выделить данных людей в одну общую категорию 3-5.
* меньше всего просрочек у людей, у которых нет детей, при этом сама категория самая многочисленная.
* больше всего просрочек у людей с 2 детьми, при этом "отрыв" от категории с 1 ребенком незначительный.
* можно сказать, что зависимости между количеством детей и возвратом кредита в срок явной нет или данных недостаточно.
* но она определенно есть между наличием детей и возвартом кредита в срок, отсуствие детей с бОльшей вероятностью гарантирует его.

### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на данный вопрос предлагаю рассмотреть и сравнить процент просрочек при разных значениях семейного положения. Сформируем сводную таблицу по общему количеству просроченных кредитов в зависимости от положения.

In [ ]:
display(family_status_dict)
data.pivot_table(index=['gender'], columns='family_status_id', values='debt', aggfunc='sum')

In [ ]:
ratio_factor('family_status_id')

**Выводы:** 
* больше всего процент просрочек у неженатых/незамужних людей, можно было бы сказать "одиноких", но...
* у вдовствующих и разведенных он как раз таки самый низкий 
* вероятно, вдовы/вдовцы и разведенные чаще все-таки более возрастные, стабильные и обязательные заемщики, а вот неженатые чаще именно молодые
* в целом, можно усмотерть зависимость, что неженатые или живущие в гражданском браке люди менее кредитоспособны, чем остальные


### Вопрос 3:Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на данный вопрос предлагаю рассмотреть и сравнить процент просрочек при разных значениях уровня доходов. Сформируем сводную таблицу по общему количеству просроченных кредитов в зависимости от категории дохода.

In [ ]:
data.pivot_table(index=['gender'], columns='income_category', values='debt', aggfunc='sum')

* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 100001 и выше — 'A'.

In [ ]:
ratio_factor('income_category')

**Выводы:** 
* ситуация по категориям `E, A, D` примерно как с количсетвом детей от 3-х, несравнимо маленькое количество данных по отношению к другим категориям, поэтому крайне сложно и не объективно делать выводы на их основе. Эти значения являются крайними значениями интервалов, в идеале для более полного исследования было бы необходимо изменить интервалы категорий для получения более равномерной наполненности.
* тем не менее, самый высокий процент по просрочке среди категории `E` - c самым низким доходом, и самый низкий процент в категории `D` - c доходом 30000-50000. Из двух наиболее часто встречаемых категорий доходов `С` и `B`, люди с более высоким доходом были более надежными заемщиками.



### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Для ответа на данный вопрос предлагаю рассмотреть и сравнить процент просрочек при разных значениях назначения кредита. Сформируем сводную таблицу по общему количеству просроченных кредитов в зависимости от цели.

In [ ]:
data.pivot_table(index=['gender'], columns='purpose_category', values='debt', aggfunc='sum')

In [ ]:
ratio_factor('purpose_category')

**Выводы:** 
* процент просрочек по кдедитам на свадьбу и недвижимость меньше, чем на автомобиль и образование и довольно значительно
* вероятно, данные цели также можно связать и с другими "переменными", скажем, кредит под цель автомобиль и образование чаще берут более молодые, неженатые, менее обеспеченные люди.

## Общий вывод:

В ходе исследования была проведена пердобработка данных, которая включала в себя:

* заполенние пропусков значений, там, где это возможно
* обработка аномалий
* удаление явных и неявных дубликатов
* категоризация
* декомпозиция исходного датафрейма
* изменение типов данных

Также были рассмотрены зависимости между целеями кредита, количеством детей, уровнем дохода и семейным положением и возвратом кредита в срок.

В итоге, можно составить "идеальный" портрет заемщика: женатый/замужняя, без детей, под недвижимость, с доходом хотя бы от 30000.

Из отдельно рассмотренных факторов, которые имеют наибольшее прогностическое значение на возврат кредита являются семейное положение и цель кредита, наименьшее - уровень дохода. Для более глубоко анализа требуется рассматривать данные факторы в сочетании между собой.
